# **Generating Taxonomic Profile from Microbiome Data**

## **1. Generating artifact**

In [5]:
##### FASTAQ to Artifact #####
# import the fasta files into a QIIME2 artifact
!qiime tools import \
  --type 'SampleData[SequencesWithQuality]' \
  --input-format SingleEndFastqManifestPhred33V2 \
  --input-path manifest.tsv \
  --output-path sequences.qza

Imported manifest.tsv as SingleEndFastqManifestPhred33V2 to sequences.qza


In [6]:
##### Artifact to Visualization #####
# QIIME to visualize our sequencing data.
!qiime demux summarize \
	--i-data sequences.qza \
	--o-visualization qualities.qzv

Saved Visualization to: qualities.qzv


In [4]:
# visualize
!qiime tools view qualities.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.Opening in existing browser session.

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

## **2. Denoise/Demultiplex data**

In [8]:
##### Quality Filtering: From Sequence to ASV ##### Change the trunc-len to 250
!qiime dada2 denoise-single \
    --i-demultiplexed-seqs sequences.qza \
    --p-trunc-len 250 \
    --p-n-threads 4 \
    --output-dir dada --verbose

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada.R --input_directory /tmp/qiime2/davo/data/356c47dc-2fe3-48f9-a044-3371720cda23/data --output_path /tmp/tmpbptgo5n3/output.tsv.biom --output_track /tmp/tmpbptgo5n3/track.tsv --filtered_directory /tmp/tmpbptgo5n3 --truncation_length 250 --trim_left 0 --max_expected_errors 2.0 --truncation_quality_score 2 --max_length Inf --pooling_method independent --chimera_method consensus --min_parental_fold 1.0 --allow_one_off False --num_threads 4 --learn_min_reads 1000000 --homopolymer_gap_penalty NULL --band_size 16

R version 4.3.3 (2024-02-29) 
Loading required package: Rcpp
DADA2: 1.30.0 / Rcpp: 1.0.13.1 / RcppParallel: 5.1.9 
2) Filtering ............................................................................................................

## **3. Generating OTU**

In [9]:
# Denoising statistics
!qiime metadata tabulate \
    --m-input-file dada/denoising_stats.qza \
    --o-visualization denoising-stats.qzv

Saved Visualization to: denoising-stats.qzv


In [10]:
# Feature table summary
!qiime feature-table summarize \
  --i-table ./dada/table.qza \
  --m-sample-metadata-file ./metadata.tsv \
  --o-visualization ./dada_freqtable.qzv

Saved Visualization to: ./dada_freqtable.qzv


In [16]:
# visualize 
!qiime tools view ./dada_freqtable.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.Opening in existing browser session.

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

## **4. Taxonomic Classification (math features to labels)**

In [ ]:
# get the classifier
!wget -nv -O \
  "silva-138-99-nb-classifier.qza" \
  "https://data.qiime2.org/classifiers/sklearn-1.4.2/silva/silva-138-99-nb-classifier.qza"

2024-12-08 05:50:58 URL:https://s3-us-west-2.amazonaws.com/qiime2-data/classifiers/sklearn-1.4.2/silva/silva-138-99-nb-classifier.qza [218245868/218245868] -> "silva-138-99-515-806-nb-classifier.qza" [1]


In [33]:
# expoert the table
!qiime tools export \
  --input-path dada/table.qza \
  --output-path exported_table

Exported dada/table.qza as BIOMV210DirFmt to directory exported_table


In [ ]:
# convert biom into csv
# !biom convert \
# 	-i exported_table/feature-table.biom \
# 	-o exported_table/otu_table.csv --to-tsv

## TRICKY NOT NECESSARY

In [31]:
# Get taxonomic OTU
!qiime feature-classifier classify-sklearn \
  --i-classifier silva-138-99-nb-classifier.qza \
  --i-reads ./dada/representative_sequences.qza \
  --o-classification taxonomy.qza

Saved FeatureData[Taxonomy] to: taxonomy.qza


In [ ]:
# convert biom into csv
!biom convert \
	-i exported_table/feature-table.biom \
	-o exported_table/otu_table.csv --to-tsv

### Extract taxonomy

In [149]:
!qiime taxa collapse \
  --i-table ./dada/table.qza  \
  --i-taxonomy taxonomy.qza \
  --p-level 6 \
  --o-collapsed-table collapsed_family_table.qza

Saved FeatureTable[Frequency] to: collapsed_family_table.qza


In [150]:
!qiime tools export \
  --input-path collapsed_family_table.qza \
  --output-path exported_family_table

Exported collapsed_family_table.qza as BIOMV210DirFmt to directory exported_family_table


In [151]:
# visualize
import biom
import pandas as pd

taxon = biom.load_table('exported_family_table/feature-table.biom')
taxon = taxon.to_dataframe()

In [152]:
taxon

,206534,206536,206538,206547,206548,206561,206562,206563,206564,206569,...,222171,224323,224324,224325,224326,224327,224328,224330,224844,224845
d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Bacteroides,13855.0,12252.0,5345.0,4.0,27.0,3807.0,5954.0,5657.0,1060.0,8001.0,...,12555.0,18068.0,27582.0,16909.0,14085.0,20334.0,23465.0,19140.0,5663.0,11811.0
d__Bacteria;p__Firmicutes;c__Clostridia;o__Oscillospirales;f__Ruminococcaceae;g__Faecalibacterium,652.0,570.0,201.0,5.0,23.0,2887.0,6126.0,7048.0,1276.0,3372.0,...,2197.0,6179.0,4676.0,5388.0,6230.0,8292.0,7873.0,3787.0,6680.0,14629.0
d__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacterales;f__Enterobacteriaceae;g__Escherichia-Shigella,1713.0,1871.0,1022.0,7127.0,2599.0,25.0,43.0,22.0,139.0,2.0,...,34.0,1926.0,2501.0,2335.0,2555.0,4727.0,4011.0,99.0,547.0,299.0
d__Bacteria;p__Firmicutes;c__Clostridia;o__Lachnospirales;f__Lachnospiraceae;g__[Ruminococcus]_torques_group,2149.0,1203.0,1052.0,43.0,19.0,0,15.0,0,0,120.0,...,84.0,1881.0,1230.0,500.0,680.0,1227.0,1708.0,182.0,86.0,142.0
d__Bacteria;p__Firmicutes;c__Clostridia;o__Lachnospirales;f__Lachnospiraceae;g__[Ruminococcus]_gnavus_group,30.0,40.0,17.0,1270.0,124.0,0,0,0,0,0,...,0,134.0,61.0,0,0,1200.0,981.0,187.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhizobiales;f__Rhizobiaceae;g__Ochrobactrum,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
d__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Caloramatoraceae;__,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
d__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Burkholderiales;f__Rhodocyclaceae;__,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.0
d__Bacteria;p__Firmicutes;c__Clostridia;o__Oscillospirales;f__Hungateiclostridiaceae;g__Saccharofermentans,0,0,0,0,0,0,0,0,0,0,...,0,0,2.0,0,0,0,0,0,0,0


In [153]:
df_split = taxon.index.str.split(';', expand=True).to_frame().reset_index(drop=True)
df_split

,0,1,2,3,4,5
0,d__Bacteria,p__Bacteroidota,c__Bacteroidia,o__Bacteroidales,f__Bacteroidaceae,g__Bacteroides
1,d__Bacteria,p__Firmicutes,c__Clostridia,o__Oscillospirales,f__Ruminococcaceae,g__Faecalibacterium
2,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Escherichia-Shigella
3,d__Bacteria,p__Firmicutes,c__Clostridia,o__Lachnospirales,f__Lachnospiraceae,g__[Ruminococcus]_torques_group
4,d__Bacteria,p__Firmicutes,c__Clostridia,o__Lachnospirales,f__Lachnospiraceae,g__[Ruminococcus]_gnavus_group
...,...,...,...,...,...,...
705,d__Bacteria,p__Proteobacteria,c__Alphaproteobacteria,o__Rhizobiales,f__Rhizobiaceae,g__Ochrobactrum
706,d__Bacteria,p__Firmicutes,c__Clostridia,o__Clostridiales,f__Caloramatoraceae,__
707,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Burkholderiales,f__Rhodocyclaceae,__
708,d__Bacteria,p__Firmicutes,c__Clostridia,o__Oscillospirales,f__Hungateiclostridiaceae,g__Saccharofermentans


### Get table for model

In [ ]:
### Combine OTU and Taxonomy data
import biom
import pandas as pd

# Load the OTU table
otu = biom.load_table('exported_table/feature-table.biom')
otu = otu.to_dataframe()

# Load the taxonomy table
taxonomy = pd.read_csv('exported_table/taxonomy.tsv', sep='\t', index_col=0)

# Merge the OTU table with the taxonomy table
otu_taxonomy_merged = pd.merge(otu, taxonomy, left_index=True, right_index=True)

# Save the merged table to a CSV file
otu_taxonomy_merged.to_csv('otu_with_taxonomy.csv')


In [ ]:
print(otu_taxonomy_merged['Confidence'].describe())

count    5350.000000
mean        0.948876
std         0.079536
min         0.315809
25%         0.927707
50%         0.992918
75%         0.999658
max         1.000000
Name: Confidence, dtype: float64


In [122]:
otu_taxonomy_merged

,206534,206536,206538,206547,206548,206561,206562,206563,206564,206569,...,224324,224325,224326,224327,224328,224330,224844,224845,Taxon,Confidence
668fdb718997fc1589c7817655d4bb5f,6.0,9.0,4.0,4.0,19.0,1699.0,3325.0,3262.0,633.0,4924.0,...,12783.0,6685.0,5904.0,10805.0,12100.0,17885.0,1708.0,6589.0,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,0.999984
a3f36ef32153f2fc2aaeac2feb23777f,482.0,509.0,159.0,0,0,750.0,2543.0,4525.0,767.0,742.0,...,4024.0,1374.0,1739.0,6074.0,6076.0,2694.0,4684.0,9331.0,d__Bacteria;p__Firmicutes;c__Clostridia;o__Osc...,0.990108
9496d87b94d90dff068f0716603930bd,11952.0,10357.0,4574.0,0,0,35.0,91.0,455.0,102.0,37.0,...,0,0,10.0,6495.0,8555.0,0,0,0,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,0.999978
1b158b8b2922d4fcad5d9cea607cbb7d,1536.0,1648.0,805.0,7117.0,2599.0,25.0,43.0,22.0,139.0,2.0,...,2495.0,2329.0,2483.0,4712.0,4008.0,99.0,547.0,299.0,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,0.908560
23fed68c6c76ab10ba1be8a43e9176e7,0,0,0,5.0,0,173.0,453.0,451.0,98.0,214.0,...,36.0,2154.0,2414.0,471.0,313.0,13.0,441.0,1034.0,d__Bacteria;p__Firmicutes;c__Clostridia;o__Osc...,0.996007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51946b237d59929b4f666bb5229fbacf,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2.0,d__Bacteria;p__Firmicutes;c__Clostridia;o__Lac...,0.765569
2707ca30f01ee8e07aa645b208068852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2.0,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,0.984180
46b10e705d5fbdc5c8d8f3a24249591e,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2.0,d__Bacteria;p__Firmicutes;c__Clostridia;o__Chr...,0.822727
4c5ce916f019b3ba5d0fa994d24aee1a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2.0,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,0.988518


In [146]:
df_split = otu_taxonomy_merged.Taxon.str.split(';', expand=True).reset_index(drop=True)
df_split

,0,1,2,3,4,5,6
0,d__Bacteria,p__Bacteroidota,c__Bacteroidia,o__Bacteroidales,f__Bacteroidaceae,g__Bacteroides,s__
1,d__Bacteria,p__Firmicutes,c__Clostridia,o__Oscillospirales,f__Ruminococcaceae,g__Faecalibacterium,s__
2,d__Bacteria,p__Bacteroidota,c__Bacteroidia,o__Bacteroidales,f__Bacteroidaceae,g__Bacteroides,s__
3,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Escherichia-Shigella,s__
4,d__Bacteria,p__Firmicutes,c__Clostridia,o__Oscillospirales,f__Ruminococcaceae,g__Faecalibacterium,s__
...,...,...,...,...,...,...,...
5345,d__Bacteria,p__Firmicutes,c__Clostridia,o__Lachnospirales,f__Lachnospiraceae,g__Roseburia,s__
5346,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Burkholderiales,f__Rhodocyclaceae,None,None
5347,d__Bacteria,p__Firmicutes,c__Clostridia,o__Christensenellales,f__Christensenellaceae,g__Christensenellaceae_R-7_group,s__
5348,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Burkholderiales,f__Sutterellaceae,g__Sutterella,s__


## **6. Alpha Rarefaction and Selecting a Rarefaction Depth**

In [17]:
# calculate rarefaction
!qiime diversity alpha-rarefaction \
  --i-table ./dada/table.qza \
  --m-metadata-file metadata.tsv \
	--p-min-depth 10 \
  --p-max-depth 4900 \
  --o-visualization alpha_rarefaction_curves.qzv

Saved Visualization to: alpha_rarefaction_curves.qzv


In [18]:
# visualize empress
!qiime tools view alpha_rarefaction_curves.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.Opening in existing browser session.

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

## **3. Phylogenetics**

In [36]:
# Aligning sequences and constructing a phylogenetic tree with QIIME2
!qiime phylogeny align-to-tree-mafft-fasttree \
	--i-sequences dada/representative_sequences.qza \
	--output-dir tree

^C

Aborted!


In [14]:
# Visualization for the tree using the empress QIIME 2 plugin
!qiime empress tree-plot \
	--i-tree tree/rooted_tree.qza \
	--o-visualization tree/empress.qzv

Saved Visualization to: tree/empress.qzv


In [35]:
# expoert the table
!qiime tools export \
  --input-path taxonomy.qza \
  --output-path exported_table

Exported taxonomy.qza as TSVTaxonomyDirectoryFormat to directory exported_table


In [15]:
# visualize empress
!qiime tools view tree/empress.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.Opening in existing browser session.

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.